In [1]:
from skillcorner.client import SkillcornerClient

import pandas as pd

import numpy as np

import os

secret_password = os.getenv("mdp_skillcorner")
client = SkillcornerClient(username = "Nathan.talbot@etu.uca.fr", password= secret_password)

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
liste_dico = [{"comp_id" : 549,
                "ranking" : ["AJ Auxerre", "Angers SCO", "AS Saint-Étienne", "Rodez Aveyron", "Paris FC", "SM Caen", "Stade Lavallois Mayenne FC",
           "Amiens Sporting Club", "En Avant de Guingamp", "Pau FC", "Grenoble Foot 38", "Girondins de Bordeaux", "SC Bastia",
           "FC Annecy", "AC Ajaccio", "Dunkerque", "ES Troyes AC", "US Quevilly-Rouen", "US Concarneau", "Valenciennes FC"],
           "annee" : "2023_2024"},
           {"comp_id" : 393,
                "ranking" : ["Le Havre AC", "FC Metz", "Girondins de Bordeaux", "SC Bastia", "SM Caen", "En Avant de Guingamp", "Paris FC",
           "AS Saint-Étienne", "FC Sochaux-Montbéliard", "Grenoble Foot 38", "US Quevilly-Rouen", "Amiens Sporting Club", "Pau FC",
           "Rodez Aveyron", "Stade Lavallois Mayenne FC", "Valenciennes FC", "FC Annecy", "Dijon FCO", "Nîmes Olympique", "Chamois Niortais FC"],
           "annee" : "2022_2023"}]

# i = 0 : année 2023/2024, i = 1 : année 2022/2023
i = 1
dico = liste_dico[i]

In [3]:
liste_match = pd.DataFrame(client.get_matches(params = {"competition_edition" : dico["comp_id"]})).id.tolist()

In [4]:
nb_journee = 5
y = [0]*20
y[:5] = [1]*5
y = y*(int(38/nb_journee) + 1)

In [5]:
# Création du Data Set
X = pd.DataFrame()


for j in range (0, len(liste_match), nb_journee*10) :
    pressure_data_json = client.get_in_possession_on_ball_pressures(params = {'competition_edition': dico["comp_id"], "group_by" : "team",
                                                                              "match" : liste_match[j : j + nb_journee*10],
                                                                     "pressure_intensity" : ["low", "medium", "high"]})
    pressure_data = pd.DataFrame(pressure_data_json).set_index("team_name")
    pressure_data.drop(["team_id", "third", "channel", "minutes_played_per_match", "adjusted_min_tip_per_match",
                   "count_match", "count_match_failed", "count_low_pressures_received_in_sample"], inplace = True, axis = 1)
    pressure_data = pressure_data.reindex(dico["ranking"])
    X = pd.concat([X, pressure_data])

X_scale = StandardScaler().fit_transform(X)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_scale, y, train_size = 0.66, random_state = 42, shuffle = True)

In [7]:
model = LogisticRegressionCV(Cs = [1, 5, 10, 20, 50, 100]).fit(X_train, y_train)

ValueError: Input X contains NaN.
LogisticRegressionCV does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
df_coef = pd.DataFrame(model.coef_.T, index = X.columns, columns = ["coeff"])
df_coef_abs = pd.DataFrame(abs(model.coef_.T), index = X.columns, columns = ["coeff en valeur absolue"])

In [ ]:
coef = pd.concat([df_coef, df_coef_abs], axis = 1).sort_values(by = "coeff en valeur absolue", ascending = False)

In [ ]:
coef.to_excel(f"..\\..\\Tableau métriques\\reg_log\\{dico["annee"]}\\Skill Corner\\reg_log_pressure.xlsx")